In [3]:
import pandas as pd
import numpy as np
import pandas_ta as pta
import ta
# from datapackage import Package
# from selenium import webdriver
# import requests
# from bs4 import BeautifulSoup
# import yfinance as yf
import os
import random
import shutil
import time
from tensorflow.keras.layers import LSTM
import tensorflow as tf
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from numpy import array
from tensorflow.keras.utils import plot_model
import graphviz
import pydot
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout

In [4]:
# Function to slice the data according to needs
def create_windows(test_set, steps):
	X_list = []
	function_set = test_set.copy()
	for i in range(len(test_set) - steps):
		x_slice = function_set[0:steps]
		X_list.append(x_slice)
		function_set = function_set.shift(-1)
	del(function_set)
	return(np.array(X_list))

In [5]:

start_time = time.time()


dropped_rows = 0
total_rows = 0
steps = 7

for folder in ["X_test", "X_train", "y_test", "y_train"]:
	for file in os.listdir("sets/" + folder):
		os.remove("sets/" + folder + "/" + file)

for file in os.listdir("models"):
	os.remove("models/" + file)	

for i in os.listdir("data"):
	ticker_code = i.replace(".csv", "")
	little_table = pd.read_csv("data/" + i)
	little_table["Close_next_day"] = pd.Series(little_table["Close"].shift(-1))
	bands = pta.bbands(close = little_table["Close"], length=20, std=2, mamode="ema")[["BBL_20_2.0", "BBU_20_2.0"]]
	little_table["lower_bb"], little_table["upper_bb"] = bands.iloc[:,0], bands.iloc[:,1]
	little_table["sma20"] = pta.sma(little_table["Close"], length=20)
	little_table["sma50"] = pta.sma(little_table["Close"], length=50)
	little_table["accdist"] = pta.ad(high=little_table["High"], low=little_table["Low"], close=little_table["Close"], volume=little_table["Volume"])
	little_table["adx"] = pta.adx(high=little_table["High"], low=little_table["Low"], close = little_table["Close"], length=30).iloc[:,0]
	little_table["aroon"] = pta.aroon(little_table["Close"], little_table["Low"], length=30).iloc[:,2]
	little_table["macd"] = pta.apo(little_table["Close"]) #apo and macd are equivalent and this function is simpler
	little_table["obv"] = pta.obv(close=little_table["Close"], volume=little_table["Volume"])
	little_table["rsi"] = pta.rsi(close=little_table["Close"], length=30)
	little_table["stoch_o"] = pta.stoch(high=little_table["High"], low=little_table["Low"], close=little_table["Close"]).iloc[:,0]
	for i in range(len(little_table)):
		little_table["Date"].iloc[i] = (pd.to_datetime(little_table["Date"].iloc[i][:10])).toordinal() #Should probably have breaken this down but it converts the string of Date to date and then to numeric (ordinal) for use with TensorFlow
		if (i % 1000 == 0):
			print(f"{i} records processed of {len(little_table)}")
	little_table["Date"] = pd.to_numeric(little_table["Date"])
	little_table["Volume"] = pd.to_numeric(little_table["Volume"])
	old_len = len(little_table)
	total_rows += old_len # counts how many rows were dropped from the original dataset
	little_table = little_table.dropna()
	dropped_rows += (old_len - len(little_table)) #counts how many rows are dropped
	features = little_table.drop(columns=["Close_next_day", ])
	labels = little_table["Close_next_day"]
	scaler = MinMaxScaler()
	X_tr, X_te, y_tr, y_te = train_test_split(features, labels, train_size=0.8, shuffle=False)
	scaler.fit_transform(X_tr)
	scaler.transform(X_te)	
	pd.DataFrame(X_tr).to_pickle("sets/X_train/" + ticker_code + "_X_train.pkl")
	pd.DataFrame(X_te).to_pickle("sets/X_test/" + ticker_code + "_X_test.pkl")
	pd.DataFrame(y_tr).to_pickle("sets/y_train/" + ticker_code + "_y_train.pkl")
	pd.DataFrame(y_te).to_pickle("sets/y_test/" + ticker_code + "_y_test.pkl")

for i in os.listdir("sets/X_train"):
	ticker_code = i.replace("_X_train.pkl", "")
	features = pd.read_pickle("sets/X_train/" + ticker_code + "_X_train.pkl")
	label = pd.read_pickle("sets/y_train/" + ticker_code + "_y_train.pkl")
	steps = 5
	X_list = create_windows(features, steps)
	n_features = X_list.shape[2]
	# define model
	model = Sequential()
	model.add(LSTM(64, activation='relu', input_shape=(steps, n_features), return_sequences=True))
	model.add(Dropout(0.2))
	model.add(LSTM(64, return_sequences=True))
	model.add(Dropout(0.2))
	model.add(LSTM(64, return_sequences=True))
	model.add(Dropout(0.2))
	model.add(LSTM(64, return_sequences=True))
	model.add(Dropout(0.2))
	model.add(LSTM(64, return_sequences=True))
	model.add(Dropout(0.2))
	model.add(LSTM(64, return_sequences=True))
	model.add(Dropout(0.2))
	model.add(LSTM(64))
	model.add(Dropout(0.2))
	model.add(Dense(units=1))
	model.compile(optimizer='adam', loss='mean_squared_error')
	model.fit(X_list, label[:(steps * -1)], epochs=20)
	X_test = pd.read_pickle("sets/X_test/" + ticker_code + "_X_test.pkl")
	y_test = pd.read_pickle("sets/y_test/" + ticker_code + "_y_test.pkl")
	X_test_win = create_windows(X_test, steps)
	score = model.evaluate(X_test_win, y_test[:(steps * -1)])
	pd.to_pickle(model, "models/" + ticker_code + "_score_" + str(round(score, 2)) + ".pkl")
print(f"Process executed in {(time.time() - start_time) / 3600} hours. {dropped_rows} rows of {total_rows} dropped due to NaN / None values ({round((dropped_rows / total_rows * 100), 2)}%).")

C:\Users\BogPoet\AppData\Local\Temp\ipykernel_27408\2509814590.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  little_table["Date"].iloc[i] = (pd.to_datetime(little_table["Date"].iloc[i][:10])).toordinal() #Should probably have breaken this down but it converts the string of Date to date and then to numeric (ordinal) for use with TensorFlow


0 records processed of 5791
1000 records processed of 5791
2000 records processed of 5791
3000 records processed of 5791
4000 records processed of 5791
5000 records processed of 5791


C:\Users\BogPoet\AppData\Local\Temp\ipykernel_27408\2509814590.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  little_table["Date"].iloc[i] = (pd.to_datetime(little_table["Date"].iloc[i][:10])).toordinal() #Should probably have breaken this down but it converts the string of Date to date and then to numeric (ordinal) for use with TensorFlow


0 records processed of 4320
1000 records processed of 4320
2000 records processed of 4320


KeyboardInterrupt: 

In [ ]:
steps = 3

for i in os.listdir("sets/X_train"):
	ticker_code = i.replace("_X_train.pkl", "")
	features = pd.read_pickle("sets/X_train/" + ticker_code + "_X_train.pkl")
	label = pd.read_pickle("sets/y_train/" + ticker_code + "_y_train.pkl")
	steps = 5
	X_list = create_windows(features, steps)
	n_features = X_list.shape[2]
	# define model
	model = Sequential()
	model.add(LSTM(64, activation='relu', input_shape=(steps, n_features), return_sequences=True))
	model.add(Dropout(0.2))
	model.add(LSTM(64, return_sequences=True))
	model.add(Dropout(0.2))
	model.add(LSTM(64, return_sequences=True))
	model.add(Dropout(0.2))
	model.add(LSTM(64, return_sequences=True))
	model.add(Dropout(0.2))
	model.add(LSTM(64, return_sequences=True))
	model.add(Dropout(0.2))
	model.add(LSTM(64, return_sequences=True))
	model.add(Dropout(0.2))
	model.add(LSTM(64))
	model.add(Dropout(0.2))
	model.add(Dense(units=1))
	model.compile(optimizer='adam', loss='mean_squared_error')
	model.fit(X_list, label[:(steps * -1)], epochs=20)
	X_test = pd.read_pickle("sets/X_test/" + ticker_code + "_X_test.pkl")
	y_test = pd.read_pickle("sets/y_test/" + ticker_code + "_y_test.pkl")
	X_test_win = create_windows(X_test, steps)
	score = model.evaluate(X_test_win, y_test[:(steps * -1)])
	pd.to_pickle(model, "models/" + ticker_code + "_score_" + str(round(score, 2)) + ".pkl")
print(f"Process executed in {(time.time() - start_time) / 3600} hours. {dropped_rows} rows of {total_rows} dropped due to NaN / None values ({round((dropped_rows / total_rows * 100), 2)}%).")

In [ ]:
test_model = pd.read_pickle("models/AVGO_score_140789.06.pkl")

Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2022-11-28 05:37:48         7852
metadata.json                                  2022-11-28 05:37:48           64
variables.h5                                   2022-11-28 05:37:48      2710000
Keras weights file (<HDF5 file "variables.h5" (mode r)>) loading:
...layers\dense
......vars
.........0
.........1
...layers\dropout
......vars
...layers\dropout_1
......vars
...layers\dropout_2
......vars
...layers\dropout_3
......vars
...layers\dropout_4
......vars
...layers\dropout_5
......vars
...layers\dropout_6
......vars
...layers\lstm
......vars
...layers\lstm\cell
......vars
.........0
.........1
.........2
...layers\lstm_1
......vars
...layers\lstm_1\cell
......vars
.........0
.........1
.........2
...layers\lstm_2
......vars
...layers\lstm_2\cell
......vars
.........0
.........1
.........2
...layers\lstm_3
......vars
...layers\lstm_

In [ ]:
model.predict(X_test)

NameError: name 'model' is not defined